In [2]:
import pandas as pd
from Bio import SeqIO
import re

# --- Cấu hình giống hệt code map ---
FASTA_PATH = r"C:\Users\dotru\STUDIE\FPTU\AiTA_Lab\Homo_sapiens.GRCh38.pep.all.fa"
CSV_PATH = r"C:\Users\dotru\STUDIE\FPTU\AiTA_Lab\data\ver2\dataset2\variant_sequence_positive.csv"

def diagnose_loss():
    # 1. Load FASTA keys
    print("Đang nạp danh sách mã Transcript từ FASTA...")
    fasta_keys = set()
    for record in SeqIO.parse(FASTA_PATH, "fasta"):
        match = re.search(r'transcript:(ENST\d+)', record.description)
        if match: fasta_keys.add(match.group(1))

    # 2. Quét CSV và đếm lỗi
    stats = {
        "Transcript_Not_In_Fasta": 0,
        "No_Protein_Position": 0,
        "Ref_AA_Mismatch": 0,
        "Invalid_AA_Format": 0,
        "Other_Errors": 0
    }
    
    print("Bắt đầu chẩn đoán...")
    chunk_size = 100000
    for chunk in pd.read_csv(CSV_PATH, chunksize=chunk_size):
        for _, row in chunk.iterrows():
            try:
                # Kiểm tra Transcript ID
                enst_id = str(row['Feature']).split('.')[0]
                if enst_id not in fasta_keys:
                    stats["Transcript_Not_In_Fasta"] += 1
                    continue

                # Kiểm tra Vị trí
                pos_str = str(row['Protein_position']).split('-')[0]
                if pos_str == '-':
                    stats["No_Protein_Position"] += 1
                    continue
                
                # Kiểm tra Amino Acid format
                aa_change = str(row['Amino_acids'])
                if '/' not in aa_change:
                    stats["Invalid_AA_Format"] += 1
                    continue

                # Nếu chạy đến đây mà vẫn mất, thì chỉ có thể là Mismatch (Không khớp Ref)
                # (Vì trong code chính ta có bước kiểm tra actual_ref_aa == ref_aa_part)
                stats["Ref_AA_Mismatch"] += 1
                
            except:
                stats["Other_Errors"] += 1

    print("\n=== BÁO CÁO CHẨN ĐOÁN MẤT MẪU ===")
    for key, value in stats.items():
        print(f"- {key}: {value:,} mẫu")
    print(f"Tổng cộng thất bại: {sum(stats.values()):,} mẫu")

diagnose_loss()

Đang nạp danh sách mã Transcript từ FASTA...
Bắt đầu chẩn đoán...

=== BÁO CÁO CHẨN ĐOÁN MẤT MẪU ===
- Transcript_Not_In_Fasta: 3,209 mẫu
- No_Protein_Position: 0 mẫu
- Ref_AA_Mismatch: 2,186,726 mẫu
- Invalid_AA_Format: 1,424,076 mẫu
- Other_Errors: 0 mẫu
Tổng cộng thất bại: 3,614,011 mẫu


In [5]:
import pandas as pd

CSV_PATH = r"C:\Users\dotru\STUDIE\FPTU\AiTA_Lab\data\ver2\dataset2\variant_sequence_positive.csv"

# Đọc thử các dòng bị coi là "Invalid Format"
print("--- Đang kiểm tra 1.4 triệu mẫu 'Invalid Format' ---")
count = 0
for chunk in pd.read_csv(CSV_PATH, chunksize=100000):
    # Tìm các dòng không có dấu '/' trong cột Amino_acids
    invalid_rows = chunk[~chunk['Amino_acids'].str.contains('/', na=False)]
    
    if not invalid_rows.empty:
        print("\nMẫu các dòng không có định dạng Ref/Alt:")
        print(invalid_rows[['Consequence', 'Protein_position', 'Amino_acids']].head(10))
        print("\nCheck Protein_position: """,invalid_rows['Protein_position'].unique())
        print("\nCheck Amino_acids: """,invalid_rows['Amino_acids'].unique())
        break # Chỉ xem 10 dòng đầu rồi dừng

--- Đang kiểm tra 1.4 triệu mẫu 'Invalid Format' ---

Mẫu các dòng không có định dạng Ref/Alt:
                                     Consequence Protein_position Amino_acids
14                            synonymous_variant               29           P
17                            synonymous_variant              184           I
21                            synonymous_variant              192           C
22                            synonymous_variant              194           C
24                            synonymous_variant              202           P
25                          splice_donor_variant                -           -
26  splice_donor_5th_base_variant,intron_variant                -           -
27                                intron_variant                -           -
28                                intron_variant                -           -
29                                intron_variant                -           -

Check Protein_position:  ['29' '184' '192' ...

In [6]:
import pandas as pd
from Bio import SeqIO
import re
import os

# --- CẤU HÌNH ---
FASTA_PATH = r"C:\Users\dotru\STUDIE\FPTU\AiTA_Lab\Homo_sapiens.GRCh38.pep.all.fa"
CSV_PATH = r"C:\Users\dotru\STUDIE\FPTU\AiTA_Lab\data\ver2\dataset2\variant_sequence_positive.csv"
OUTPUT_PATH = r"C:\Users\dotru\STUDIE\FPTU\AiTA_Lab\data\ver2\dataset2\variant_protein_sequence_101aa.csv"

WINDOW = 50 
TARGET_LEN = 101 # 50 + 1 (đột biến) + 50
PAD_CHAR = 'X'

In [7]:
# --- HÀM TIỆN ÍCH ---

def load_ensembl_protein_fasta(path):
    """
    Kế thừa logic load genome: Map từ Transcript ID (ENST) sang Protein Sequence.
    """
    mapping = {}
    print(f"🧬 Đang nạp FASTA từ {path}...")
    for record in SeqIO.parse(path, "fasta"):
        # Trích xuất ENST từ header: ... transcript:ENST00000641515.2 ...
        match = re.search(r'transcript:(ENST\d+)', record.description)
        if match:
            enst_id = match.group(1)
            mapping[enst_id] = str(record.seq).upper()
    print(f"✅ Đã nạp {len(mapping)} mã transcript.")
    return mapping

def normalize_centered_protein(seq, center_idx, target_len, pad_char='X'):
    """
    Kế thừa hoàn toàn logic 'Symmetric Crop/Pad' từ notebook DNA của bạn.
    """
    half = target_len // 2
    start = center_idx - half
    end = center_idx + half + 1
    
    pad_left = max(0, -start)
    pad_right = max(0, end - len(seq))
    
    crop_left = max(0, start)
    crop_right = min(len(seq), end)
    
    final_seq = (pad_char * pad_left) + seq[crop_left:crop_right] + (pad_char * pad_right)
    return final_seq[:target_len] # Đảm bảo luôn đủ 101

In [8]:
# --- LOGIC XỬ LÝ CHÍNH ---

# 1. Load FASTA
protein_dict = load_ensembl_protein_fasta(FASTA_PATH)

# 2. Đọc CSV theo chunk (cho 3 triệu dòng)
print(f"📊 Đang xử lý file CSV: {CSV_PATH}")
chunk_size = 100000
first_chunk = True

for chunk in pd.read_csv(CSV_PATH, chunksize=chunk_size):
    results = []
    
    for idx, row in chunk.iterrows():
        try:
            # Trích xuất Transcript ID (Feature)
            enst_id = str(row['Feature']).split('.')[0]
            if enst_id not in protein_dict:
                results.append((None, None))
                continue
            
            full_ref_protein = protein_dict[enst_id]
            
            # Xử lý vị trí (Lấy số đầu tiên nếu là dải 226-228)
            pos_str = str(row['Protein_position']).split('-')[0]
            if pos_str == '-':
                results.append((None, None))
                continue
            pos_1based = int(pos_str)
            pos_0based = pos_1based - 1
            
            # Xử lý Amino Acid (Ref/Alt)
            aa_change = str(row['Amino_acids']).strip()
            
            if '/' in aa_change:
                # Trường hợp Missense, Nonsense, Frameshift (Ví dụ: R/Q, R/*, RTP/LLLX)
                ref_aa_part, alt_aa_part = aa_change.split('/')
            elif aa_change != '-' and len(aa_change) > 0:
                # Trường hợp Synonymous (Ví dụ: P, I, LL)
                # Đột biến đồng nghĩa nên Ref và Alt giống hệt nhau
                ref_aa_part = aa_change
                alt_aa_part = aa_change
            else:
                # Bỏ qua nếu là dấu '-' (Intron/Non-coding)
                results.append((None, None))
                continue
            
            # --- XÁC MINH (Kế thừa verify_ref_seq_center) ---
            actual_ref_aa = full_ref_protein[pos_0based : pos_0based + len(ref_aa_part)]
            if actual_ref_aa != ref_aa_part:
                # Nếu không khớp tuyệt đối, đánh dấu lỗi hoặc bỏ qua
                results.append((None, None))
                continue

            # --- TẠO CHUỖI ALT ---
            # Thay thế đoạn ref bằng alt
            alt_full_protein = (full_ref_protein[:pos_0based] + 
                                alt_aa_part + 
                                full_ref_protein[pos_0based + len(ref_aa_part):])
            
            # XỬ LÝ NONSENSE/STOP: Nếu gặp * hoặc X trong phần Alt
            # Biến tất cả phía sau điểm dừng thành PAD_CHAR 'X'
            if '*' in alt_full_protein or 'X' in alt_aa_part:
                stop_idx = alt_full_protein.find('*')
                if stop_idx == -1: stop_idx = alt_full_protein.find('X', pos_0based)
                # Giữ lại ký tự dừng, còn lại phía sau là X
                alt_full_protein = alt_full_protein[:stop_idx+1] + (PAD_CHAR * (len(full_ref_protein)))

            # --- CẮT CỬA SỔ 101 AA ---
            ref_101 = normalize_centered_protein(full_ref_protein, pos_0based, TARGET_LEN, PAD_CHAR)
            alt_101 = normalize_centered_protein(alt_full_protein, pos_0based, TARGET_LEN, PAD_CHAR)
            
            results.append((ref_101, alt_101))
            
        except:
            results.append((None, None))

    # Gán kết quả vào chunk
    chunk['prot_ref_seq'], chunk['prot_alt_seq'] = zip(*results)
    
    # Lưu xuống file (Append mode)
    chunk.dropna(subset=['prot_ref_seq']).to_csv(
        OUTPUT_PATH, 
        mode='a' if not first_chunk else 'w', 
        index=False, 
        header=first_chunk
    )
    first_chunk = False
    print(f" > Đã lưu xong một cụm dữ liệu...")

print(f"🎉 Hoàn thành! File kết quả: {OUTPUT_PATH}")

🧬 Đang nạp FASTA từ C:\Users\dotru\STUDIE\FPTU\AiTA_Lab\Homo_sapiens.GRCh38.pep.all.fa...
✅ Đã nạp 245535 mã transcript.
📊 Đang xử lý file CSV: C:\Users\dotru\STUDIE\FPTU\AiTA_Lab\data\ver2\dataset2\variant_sequence_positive.csv
 > Đã lưu xong một cụm dữ liệu...
 > Đã lưu xong một cụm dữ liệu...
 > Đã lưu xong một cụm dữ liệu...
 > Đã lưu xong một cụm dữ liệu...
 > Đã lưu xong một cụm dữ liệu...
 > Đã lưu xong một cụm dữ liệu...
 > Đã lưu xong một cụm dữ liệu...
 > Đã lưu xong một cụm dữ liệu...
 > Đã lưu xong một cụm dữ liệu...
 > Đã lưu xong một cụm dữ liệu...
 > Đã lưu xong một cụm dữ liệu...
 > Đã lưu xong một cụm dữ liệu...
 > Đã lưu xong một cụm dữ liệu...
 > Đã lưu xong một cụm dữ liệu...
 > Đã lưu xong một cụm dữ liệu...
 > Đã lưu xong một cụm dữ liệu...
 > Đã lưu xong một cụm dữ liệu...
 > Đã lưu xong một cụm dữ liệu...
 > Đã lưu xong một cụm dữ liệu...
 > Đã lưu xong một cụm dữ liệu...
 > Đã lưu xong một cụm dữ liệu...
 > Đã lưu xong một cụm dữ liệu...
 > Đã lưu xong một cụm 

In [10]:
import pandas as pd
import numpy as np

# 1. Đọc file CSV
# Thay đường dẫn bên dưới bằng file của bạn
file_path = r"C:\Users\dotru\STUDIE\FPTU\AiTA_Lab\data\ver2\dataset2\variant_protein_sequence_101aa.csv"
df = pd.read_csv(file_path)

# Danh sách giá trị cần kiểm tra
invalid_values = ["-", "na", "n/a", "NA", "N/A", "NaN", "nan"]

# Thay thế toàn bộ giá trị lỗi trong toàn DataFrame bằng np.nan
df = df.replace(invalid_values, np.nan)

print("=== 1. Xem sơ bộ vài dòng đầu ===")
print(df.head(), "\n")

print("=== 2. Thông tin tổng quan (số dòng, kiểu dữ liệu, non-null) ===")
print(df.info(), "\n")

print("=== 3. Thống kê mô tả cho cột số ===")
print(df.describe(), "\n")

print("=== 4. Kiểm tra số lượng giá trị thiếu (NaN) theo từng cột ===")
print(df.isnull().sum(), "\n")

print("=== 5. Kiểm tra trùng lặp dòng ===")
num_duplicates = df.duplicated().sum()
print(f"Số dòng trùng lặp: {num_duplicates}\n")

print("=== 6. Thống kê số lượng giá trị duy nhất cho từng cột ===")
print(df.nunique(), "\n")

lengths = df['prot_ref_seq'].str.len().unique()
print(f"=== 7. Các độ dài chuỗi hiện có: {lengths} ===") 

check_diff = df[df['prot_ref_seq'].str[50] == df['prot_alt_seq'].str[50]]
print(f"=== 8. Số dòng có tâm giống nhau (có thể là synonymous): {len(check_diff)} ===")


=== 1. Xem sơ bộ vài dòng đầu ===
   AlleleID                       Type  \
0   2193183  single nucleotide variant   
1   4039319  single nucleotide variant   
2   3374047  single nucleotide variant   
3   4039320  single nucleotide variant   
4   3374048  single nucleotide variant   

                                           Name  GeneID GeneSymbol  \
0   NM_001005484.2(OR4F5):c.107A>G (p.Glu36Gly)   79501      OR4F5   
1   NM_001005484.2(OR4F5):c.281A>G (p.Lys94Arg)   79501      OR4F5   
2   NM_001005484.2(OR4F5):c.287T>G (p.Ile96Ser)   79501      OR4F5   
3  NM_001005484.2(OR4F5):c.377T>C (p.Met126Thr)   79501      OR4F5   
4  NM_001005484.2(OR4F5):c.396G>A (p.Met132Ile)   79501      OR4F5   

      HGNC_ID    ClinicalSignificance  ClinSigSimple     PhenotypeIDS  \
0  HGNC:14825           Likely benign              0  MedGen:CN169374   
1  HGNC:14825  Uncertain significance              0  MedGen:CN169374   
2  HGNC:14825  Uncertain significance              0  MedGen:CN169374   


In [ ]:
import pandas as pd

file_path = r"C:\Users\dotru\STUDIE\FPTU\AiTA_Lab\data\ver2\dataset1\variant_official_final_clean.csv"
output_path = r"C:\Users\dotru\STUDIE\FPTU\AiTA_Lab\data\ver2\dataset1\variant_official_final_clean.parquet"
df = pd.read_csv(file_path)
df.to_parquet(output_path, index = False)
print('Convert thành công variant_official_final_clean từ csv sang parquet')

Convert thành công variant_official_final_clean từ csv sang parquet


In [2]:
df = pd.read_parquet(r"C:\Users\dotru\STUDIE\FPTU\AiTA_Lab\data\ver2\dataset1\variant_official_final_clean.parquet")
df

,AlleleID,Type,Name,GeneID,GeneSymbol,HGNC_ID,ClinicalSignificance,ClinSigSimple,PhenotypeIDS,PhenotypeList,...,Assembly,ChromosomeAccession,CHROM,Start,Stop,ReviewStatus,VariationID,POS,REF,ALT
0,3544463,Deletion,NM_001005484.2(OR4F5):c.9+1354del,79501,OR4F5,HGNC:14825,Uncertain significance,0,"Human Phenotype Ontology:HP:0000547,MONDO:MOND...",Retinitis pigmentosa,...,GRCh38,NC_000001.11,chr1,66927,66927,"criteria provided, single submitter",3385321,66926,AG,A
1,2193183,single nucleotide variant,NM_001005484.2(OR4F5):c.107A>G (p.Glu36Gly),79501,OR4F5,HGNC:14825,Likely benign,0,MedGen:CN169374,not specified,...,GRCh38,NC_000001.11,chr1,69134,69134,"criteria provided, single submitter",2205837,69134,A,G
2,4039319,single nucleotide variant,NM_001005484.2(OR4F5):c.281A>G (p.Lys94Arg),79501,OR4F5,HGNC:14825,Uncertain significance,0,MedGen:CN169374,not specified,...,GRCh38,NC_000001.11,chr1,69308,69308,"criteria provided, single submitter",3925305,69308,A,G
3,3374047,single nucleotide variant,NM_001005484.2(OR4F5):c.287T>G (p.Ile96Ser),79501,OR4F5,HGNC:14825,Uncertain significance,0,MedGen:CN169374,not specified,...,GRCh38,NC_000001.11,chr1,69314,69314,"criteria provided, single submitter",3205580,69314,T,G
4,4039320,single nucleotide variant,NM_001005484.2(OR4F5):c.377T>C (p.Met126Thr),79501,OR4F5,HGNC:14825,Uncertain significance,0,MedGen:CN169374,not specified,...,GRCh38,NC_000001.11,chr1,69404,69404,"criteria provided, single submitter",3925306,69404,T,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3875326,706348,single nucleotide variant,NM_002186.3(IL9R):c.991G>A (p.Gly331Arg),3581,IL9R,HGNC:6030,Benign,0,MedGen:C3661900,not provided,...,GRCh38,NC_000024.10,chrY,57196354,57196354,"criteria provided, multiple submitters, no con...",769193,57196354,G,A
3875327,706349,single nucleotide variant,NM_002186.3(IL9R):c.1094G>A (p.Arg365His),3581,IL9R,HGNC:6030,Benign,0,MedGen:C3661900,not provided,...,GRCh38,NC_000024.10,chrY,57196457,57196457,"criteria provided, multiple submitters, no con...",771064,57196457,G,A
3875328,4197692,single nucleotide variant,NC_000023.11:g.156022415C>A,-1,None,None,Likely benign,0,MedGen:C3661900,not provided,...,GRCh38,NC_000024.10,chrY,57208935,57208935,"criteria provided, single submitter",4085007,57208935,C,A
3875329,3184976,single nucleotide variant,NC_000023.11:g.156023032T>C,-1,None,None,Likely benign,0,MedGen:C3661900,not provided,...,GRCh38,NC_000024.10,chrY,57209552,57209552,"criteria provided, single submitter",3024664,57209552,T,C


In [ ]:
import pandas as pd

file_path = r"C:\Users\dotru\STUDIE\FPTU\AiTA_Lab\data\ver2\dataset2\variant_sequence_positive.csv"
output_path = r"C:\Users\dotru\STUDIE\FPTU\AiTA_Lab\data\ver2\dataset2\variant_sequence_positive.parquet"
df = pd.read_csv(file_path)
df.to_parquet(output_path, index = False)
print('Convert thành công variant_sequence_positive từ csv sang parquet')

Convert thành công variant_sequence_positive từ csv sang parquet


In [3]:
df = pd.read_parquet(r"C:\Users\dotru\STUDIE\FPTU\AiTA_Lab\data\ver2\dataset2\variant_sequence_positive.parquet")
df

,AlleleID,Type,Name,GeneID,GeneSymbol,HGNC_ID,ClinicalSignificance,ClinSigSimple,PhenotypeIDS,PhenotypeList,...,Consequence,HGVSc,Feature,Protein_id,Protein_position,Amino_acids,Codons,canonical_center,ref_seq,alt_seq
0,2193183,single nucleotide variant,NM_001005484.2(OR4F5):c.107A>G (p.Glu36Gly),79501,OR4F5,HGNC:14825,Likely benign,0,MedGen:CN169374,not specified,...,missense_variant,ENST00000641515.2:c.107A>G,ENST00000641515,-,36,E/G,gAa/gGa,69134,GAGACACTGAGAAGCCGAGATAACTGAATTATAAGGCATAGCCAGG...,GAGACACTGAGAAGCCGAGATAACTGAATTATAAGGCATAGCCAGG...
1,4039319,single nucleotide variant,NM_001005484.2(OR4F5):c.281A>G (p.Lys94Arg),79501,OR4F5,HGNC:14825,Uncertain significance,0,MedGen:CN169374,not specified,...,missense_variant,ENST00000641515.2:c.281A>G,ENST00000641515,-,94,K/R,aAg/aGg,69308,CTGACTTCCTTCTCCTTCTCTTCTTCAAGGTAACTGCAGAGGCTAT...,CTGACTTCCTTCTCCTTCTCTTCTTCAAGGTAACTGCAGAGGCTAT...
2,3374047,single nucleotide variant,NM_001005484.2(OR4F5):c.287T>G (p.Ile96Ser),79501,OR4F5,HGNC:14825,Uncertain significance,0,MedGen:CN169374,not specified,...,missense_variant,ENST00000641515.2:c.287T>G,ENST00000641515,-,96,I/S,aTt/aGt,69314,TCCTTCTCCTTCTCTTCTTCAAGGTAACTGCAGAGGCTATTTCCTG...,TCCTTCTCCTTCTCTTCTTCAAGGTAACTGCAGAGGCTATTTCCTG...
3,4039320,single nucleotide variant,NM_001005484.2(OR4F5):c.377T>C (p.Met126Thr),79501,OR4F5,HGNC:14825,Uncertain significance,0,MedGen:CN169374,not specified,...,missense_variant,ENST00000641515.2:c.377T>C,ENST00000641515,-,126,M/T,aTg/aCg,69404,TCATTTTTCTGGGTCTCTCTGATTCTCAGGAACTCCAGACCTTCCT...,TCATTTTTCTGGGTCTCTCTGATTCTCAGGAACTCCAGACCTTCCT...
4,3374048,single nucleotide variant,NM_001005484.2(OR4F5):c.396G>A (p.Met132Ile),79501,OR4F5,HGNC:14825,Uncertain significance,0,MedGen:CN169374,not specified,...,missense_variant,ENST00000641515.2:c.396G>A,ENST00000641515,-,132,M/I,atG/atA,69423,TGATTCTCAGGAACTCCAGACCTTCCTATTTATGTTGTTTTTTGTA...,TGATTCTCAGGAACTCCAGACCTTCCTATTTATGTTGTTTTTTGTA...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3614006,706348,single nucleotide variant,NM_002186.3(IL9R):c.991G>A (p.Gly331Arg),3581,IL9R,HGNC:6030,Benign,0,MedGen:C3661900,not provided,...,missense_variant,ENST00000711289.2:c.991G>A,ENST00000711289,-,331,G/R,Ggg/Agg,57196354,CATCTGTACCTCGGTTCAGGCTGCCGTGGGCACCAGGCCCTGTGCT...,CATCTGTACCTCGGTTCAGGCTGCCGTGGGCACCAGGCCCTGTGCT...
3614007,706349,single nucleotide variant,NM_002186.3(IL9R):c.1094G>A (p.Arg365His),3581,IL9R,HGNC:6030,Benign,0,MedGen:C3661900,not provided,...,missense_variant,ENST00000711289.2:c.1094G>A,ENST00000711289,-,365,R/H,cGt/cAt,57196457,ACATGGAGGAAAGACAGAATGTCCAAGACACAGGCGCTGCTTGGCC...,ACATGGAGGAAAGACAGAATGTCCAAGACACAGGCGCTGCTTGGCC...
3614008,4197692,single nucleotide variant,NC_000023.11:g.156022415C>A,-1,None,None,Likely benign,0,MedGen:C3661900,not provided,...,synonymous_variant,ENST00000711285.2:c.540C>A,ENST00000711285,-,180,I,atC/atA,57208935,GCACCGCCCCCTGGACGAGCGGGCCCTGCAGGTCTGCTGGCTGCGC...,GCACCGCCCCCTGGACGAGCGGGCCCTGCAGGTCTGCTGGCTGCGC...
3614009,3184976,single nucleotide variant,NC_000023.11:g.156023032T>C,-1,None,None,Likely benign,0,MedGen:C3661900,not provided,...,synonymous_variant,ENST00000711285.2:c.741T>C,ENST00000711285,-,247,Y,taT/taC,57209552,TGGTGCTGTAACAAAGACCCATGTGATGCTGGGGGCAGAGACAGAG...,TGGTGCTGTAACAAAGACCCATGTGATGCTGGGGGCAGAGACAGAG...


In [ ]:
import pandas as pd

file_path = r"C:\Users\dotru\STUDIE\FPTU\AiTA_Lab\data\ver2\dataset2\variant_protein_sequence_101aa.csv"
output_path = r"C:\Users\dotru\STUDIE\FPTU\AiTA_Lab\data\ver2\dataset2\variant_protein_sequence_101aa.parquet"
df = pd.read_csv(file_path)
df.to_parquet(output_path, index = False)
print('Convert thành công variant_protein_sequence_101aa từ csv sang parquet')

Convert thành công variant_protein_sequence_101aa từ csv sang parquet


In [2]:
df = pd.read_parquet(r"C:\Users\dotru\STUDIE\FPTU\AiTA_Lab\data\ver2\dataset2\variant_protein_sequence_101aa.parquet")
df

,AlleleID,Type,Name,GeneID,GeneSymbol,HGNC_ID,ClinicalSignificance,ClinSigSimple,PhenotypeIDS,PhenotypeList,...,Feature,Protein_id,Protein_position,Amino_acids,Codons,canonical_center,ref_seq,alt_seq,prot_ref_seq,prot_alt_seq
0,2193183,single nucleotide variant,NM_001005484.2(OR4F5):c.107A>G (p.Glu36Gly),79501,OR4F5,HGNC:14825,Likely benign,0,MedGen:CN169374,not specified,...,ENST00000641515,-,36,E/G,gAa/gGa,69134,GAGACACTGAGAAGCCGAGATAACTGAATTATAAGGCATAGCCAGG...,GAGACACTGAGAAGCCGAGATAACTGAATTATAAGGCATAGCCAGG...,XXXXXXXXXXXXXXXMKKVTAEAISWNESTSETNNSMVTEFIFLGL...,XXXXXXXXXXXXXXXMKKVTAEAISWNESTSETNNSMVTEFIFLGL...
1,4039319,single nucleotide variant,NM_001005484.2(OR4F5):c.281A>G (p.Lys94Arg),79501,OR4F5,HGNC:14825,Uncertain significance,0,MedGen:CN169374,not specified,...,ENST00000641515,-,94,K/R,aAg/aGg,69308,CTGACTTCCTTCTCCTTCTCTTCTTCAAGGTAACTGCAGAGGCTAT...,CTGACTTCCTTCTCCTTCTCTTCTTCAAGGTAACTGCAGAGGCTAT...,LFFVFYGGIVFGNLLIVITVVSDSHLHSPMYFLLANLSLIDLSLSS...,LFFVFYGGIVFGNLLIVITVVSDSHLHSPMYFLLANLSLIDLSLSS...
2,3374047,single nucleotide variant,NM_001005484.2(OR4F5):c.287T>G (p.Ile96Ser),79501,OR4F5,HGNC:14825,Uncertain significance,0,MedGen:CN169374,not specified,...,ENST00000641515,-,96,I/S,aTt/aGt,69314,TCCTTCTCCTTCTCTTCTTCAAGGTAACTGCAGAGGCTATTTCCTG...,TCCTTCTCCTTCTCTTCTTCAAGGTAACTGCAGAGGCTATTTCCTG...,FVFYGGIVFGNLLIVITVVSDSHLHSPMYFLLANLSLIDLSLSSVT...,FVFYGGIVFGNLLIVITVVSDSHLHSPMYFLLANLSLIDLSLSSVT...
3,4039320,single nucleotide variant,NM_001005484.2(OR4F5):c.377T>C (p.Met126Thr),79501,OR4F5,HGNC:14825,Uncertain significance,0,MedGen:CN169374,not specified,...,ENST00000641515,-,126,M/T,aTg/aCg,69404,TCATTTTTCTGGGTCTCTCTGATTCTCAGGAACTCCAGACCTTCCT...,TCATTTTTCTGGGTCTCTCTGATTCTCAGGAACTCCAGACCTTCCT...,LLANLSLIDLSLSSVTAPKMITDFFSQRKVISFKGCLVQIFLLHFF...,LLANLSLIDLSLSSVTAPKMITDFFSQRKVISFKGCLVQIFLLHFF...
4,3374048,single nucleotide variant,NM_001005484.2(OR4F5):c.396G>A (p.Met132Ile),79501,OR4F5,HGNC:14825,Uncertain significance,0,MedGen:CN169374,not specified,...,ENST00000641515,-,132,M/I,atG/atA,69423,TGATTCTCAGGAACTCCAGACCTTCCTATTTATGTTGTTTTTTGTA...,TGATTCTCAGGAACTCCAGACCTTCCTATTTATGTTGTTTTTTGTA...,LIDLSLSSVTAPKMITDFFSQRKVISFKGCLVQIFLLHFFGGSEMV...,LIDLSLSSVTAPKMITDFFSQRKVISFKGCLVQIFLLHFFGGSEMV...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2837610,706348,single nucleotide variant,NM_002186.3(IL9R):c.991G>A (p.Gly331Arg),3581,IL9R,HGNC:6030,Benign,0,MedGen:C3661900,not provided,...,ENST00000711289,-,331,G/R,Ggg/Agg,57196354,CATCTGTACCTCGGTTCAGGCTGCCGTGGGCACCAGGCCCTGTGCT...,CATCTGTACCTCGGTTCAGGCTGCCGTGGGCACCAGGCCCTGTGCT...,LLLTGPTYLLFKLSPRVKRIFYQNVPSPAMFFQPLYSVHNGNFQTW...,LLLTGPTYLLFKLSPRVKRIFYQNVPSPAMFFQPLYSVHNGNFQTW...
2837611,706349,single nucleotide variant,NM_002186.3(IL9R):c.1094G>A (p.Arg365His),3581,IL9R,HGNC:6030,Benign,0,MedGen:C3661900,not provided,...,ENST00000711289,-,365,R/H,cGt/cAt,57196457,ACATGGAGGAAAGACAGAATGTCCAAGACACAGGCGCTGCTTGGCC...,ACATGGAGGAAAGACAGAATGTCCAAGACACAGGCGCTGCTTGGCC...,LYSVHNGNFQTWMGAHGAGVLLSQDCAGTPQGALEPCVQEATALLT...,LYSVHNGNFQTWMGAHGAGVLLSQDCAGTPQGALEPCVQEATALLT...
2837612,4197692,single nucleotide variant,NC_000023.11:g.156022415C>A,-1,None,None,Likely benign,0,MedGen:C3661900,not provided,...,ENST00000711285,-,180,I,atC/atA,57208935,GCACCGCCCCCTGGACGAGCGGGCCCTGCAGGTCTGCTGGCTGCGC...,GCACCGCCCCCTGGACGAGCGGGCCCTGCAGGTCTGCTGGCTGCGC...,RRPRHRIQSKHRPLDERALQEKLKDFPVCVSTKPEPEDDAEEGLGG...,RRPRHRIQSKHRPLDERALQEKLKDFPVCVSTKPEPEDDAEEGLGG...
2837613,3184976,single nucleotide variant,NC_000023.11:g.156023032T>C,-1,None,None,Likely benign,0,MedGen:C3661900,not provided,...,ENST00000711285,-,247,Y,taT/taC,57209552,TGGTGCTGTAACAAAGACCCATGTGATGCTGGGGGCAGAGACAGAG...,TGGTGCTGTAACAAAGACCCATGTGATGCTGGGGGCAGAGACAGAG...,KKYVFLDPLAGAVTKTHVMLGAETEEKLFDAPLSISKREQLEQQVP...,KKYVFLDPLAGAVTKTHVMLGAETEEKLFDAPLSISKREQLEQQVP...


In [7]:
dict = df['ClinicalSignificance'].value_counts().to_dict()
dict

{'Uncertain significance': 1839392,
 'Likely benign': 620776,
 'Conflicting classifications of pathogenicity': 123304,
 'Benign': 70113,
 'Pathogenic': 69236,
 'Likely pathogenic': 45925,
 'Benign/Likely benign': 44432,
 'Pathogenic/Likely pathogenic': 18146,
 'not provided': 3804,
 'drug response': 515,
 'other': 513,
 'no classification for the single variant': 378,
 'risk factor': 183,
 'Uncertain significance/Uncertain risk allele': 123,
 'association': 110,
 'no classifications from unflagged records': 96,
 'Affects': 88,
 'Pathogenic; other': 66,
 'Likely risk allele': 53,
 'Likely pathogenic, low penetrance': 42,
 'Pathogenic; drug response': 39,
 'Conflicting classifications of pathogenicity; other': 19,
 'protective': 16,
 'Uncertain risk allele': 16,
 'Pathogenic, low penetrance': 15,
 'Benign; risk factor': 14,
 'Likely pathogenic; drug response': 13,
 'Conflicting classifications of pathogenicity; risk factor': 13,
 'Likely pathogenic/Likely risk allele': 12,
 'Benign; othe